In [1]:
import os
import uuid
import logging
import subprocess
from pathlib import Path
from dotenv import load_dotenv

from utils import (
    extract_pdf_content_streams,
    build_pdf_prompt,
    call_llm_generate_streams,
    build_full_pdf_from_streams
)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv(".env")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")
LLM_NAME = os.getenv("LLM")

# --- Helper: Decompress PDF using qpdf ---
def uncompress_pdf(input_pdf_path: Path, output_pdf_path: Path):
    logger.info("🔓 Uncompressing: %s", input_pdf_path)
    subprocess.run([
        'qpdf', '--qdf', '--object-streams=disable',
        str(input_pdf_path), str(output_pdf_path)
    ], check=True)
    logger.info("💾 Saved uncompressed file to: %s", output_pdf_path)

# --- Main processing function ---
def process_pdf_for_accessibility(input_pdf_path, metadata="{}", output_dir="/tmp", retries=2):
    logger.info("📄 Processing file: %s", input_pdf_path)

    input_path = Path(input_pdf_path)
    uncompressed_path = Path(output_dir) / f"{input_path.stem}_uncompressed{input_path.suffix}"

    uncompress_pdf(input_path, uncompressed_path)

    logger.info("🔍 Extracting PDF content streams...")
    content_streams = extract_pdf_content_streams(str(uncompressed_path))
    logger.info("📄 Extracted %d content stream(s)", len(content_streams))

    logger.info("🧾 Building LLM prompt")
    prompt_lines = [
        "You are a PDF cleaner. Your task is to rewrite PDF content streams for accessibility.",
        "Each input is a low-level PDF page content stream (BT/ET block).",
        "For each stream, return an improved version using Helvetica font, 12pt size, and correctly positioned text.",
        "Ensure the syntax remains valid PDF content streams with BT/ET blocks.",
        "Respond only with updated PDF streams — no explanation or formatting.",
        "Each stream must be returned between `<< /Length L >>` and `endstream`, where L is the byte length of the stream body.",
        "Do not wrap the response in code fences or markdown."
    ]
    prompt = build_pdf_prompt(prompt_lines, content_streams)

    logger.debug("📨 Prompt preview:\n%s", prompt[:500])
    logger.info("🔁 Calling LLM with up to %d retries", retries)
    improved_streams = call_llm_generate_streams(
        prompt, model=LLM_NAME, base_url=OLLAMA_BASE_URL, retries=retries
    )
    logger.info("✅ LLM returned %d improved stream(s)", len(improved_streams))

    output_path = Path(output_dir) / f"accessible_{input_path.name}"
    logger.info("🛠️  Building PDF to: %s", output_path)
    build_full_pdf_from_streams(improved_streams, str(output_path))

    return str(output_path)

# --- Batch directory processor ---
def process_pdf_directory(input_dir, output_dir, retries=2):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".pdf"):
            input_path = os.path.join(input_dir, filename)
            try:
                output_path = process_pdf_for_accessibility(input_path, output_dir=output_dir, retries=retries)
                print(f"✅ Processed: {filename} → {output_path}")
            except Exception as e:
                print(f"❌ Failed: {filename} — {e}")

# 🔧 Define input/output
INPUT_DIR = "./ipynb-input"
OUTPUT_DIR = "./ipynb-output"

# 🏃 Run batch processing
process_pdf_directory(INPUT_DIR, OUTPUT_DIR)

ModuleNotFoundError: No module named 'utils'